<a href="https://colab.research.google.com/github/mahshadmoradi/Reinforcement-Learning/blob/main/RL_HW3_On_Policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Mahshad Moradi

Student ID: 400109373

In this assignment, we will implement and test REINFORCE and PPO, which are both on-policy RL algortihms.

# REINFORCE algorithm **(40 points)**

## Setup

We must first install the required packages.

In [11]:
!pip -q install gymnasium[mujoco]
!pip install imageio -q

In [12]:
import gymnasium as gym
import random
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
from collections import namedtuple, deque
import imageio

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions import Categorical

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Explore the environment

We will train an REINFORCE agent on the `CartPole` environment.

This code displays a video given it's path.

In [13]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

Explore the `CartPole` environment using random actions. At each timestep, render the current frame, and use it to make a video of the trajectory.

In [17]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
frames = []

env.reset()
for _ in range(100):
    frames.append(env.render())
    action = env.action_space.sample() # TODO: select a random action
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()
imageio.mimsave('./CartPole.mp4', frames, fps=25)
show_video('./CartPole.mp4')


## Policy Network **(10 points)**

Complete the following code to build an agent that predicts the the probability of playing each action, given the state.

In [18]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        # TODO
        # Define the Policy Network architecture
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer

    def forward(self, x):
        # TODO
        # predict the probability of playing each action
        x = F.relu(self.fc1(x))
        action_probs = F.softmax(self.fc2(x), dim=1)
        return action_probs

## Agent **(20 points)**

REINFORCE algorithm works by interacting with an environment by taking actions based on a policy. As the agent collects rewards from the environment, it records the outcomes and the **log probabilities** of the actions it took. At the end of an episode, the algorithm calculates the total **discounted reward** from each step—this is known as the return.

$$ R_t = \sum_{k=t}^{T} \gamma^{k-t} r_k
 $$

These returns are used to weight the logged probabilities, actions that lead to higher returns are made more probable.


$$ \theta \leftarrow \theta + \alpha \sum_{t=0}^{T-1} \gamma^t R_t \nabla_\theta \log \pi_\theta(a_t|s_t)
 $$


In [19]:
class REINFORCEAgent:
    def __init__(self, policy, optimizer, gamma=0.99):
        self.policy = policy
        self.optimizer = optimizer
        self.gamma = gamma
        self.log_probs = []
        self.rewards = []

    def select_action(self, state):
        # TODO
        # select an action by sampling from the actor's response
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.policy(state)
        m = torch.distributions.Categorical(probs)
        action = m.sample()
        self.log_probs.append(m.log_prob(action))
        action = action.item()
        return action

    def update_policy(self):
        R = 0
        policy_loss = []
        returns = []

        # TODO
        # Calculate the discounted reward
        # according to the given definition :
        for r in self.rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)

        returns = torch.tensor(returns)

        # TODO
        # Calculate the policy loss
        for log_prob, R in zip(self.log_probs, returns):
            policy_loss.append(-log_prob * R)


        self.optimizer.zero_grad()
        policy_loss = torch.cat(policy_loss).sum() # TODO
        policy_loss.backward()
        self.optimizer.step()

        # Reset the rewards and log probabilities
        del self.rewards[:]
        del self.log_probs[:]

    def store_reward(self, reward):
        self.rewards.append(reward)


## Training **(5 points)**

Define the hyperparameters and complete the training loop.

In [ ]:
env = gym.make('CartPole-v1')
input_size = env.observation_space.shape[0]  # The size of the observation space # TODO
output_size = env.action_space.n  # The number of actions available # TODO
lr = 5e-3 # TODO

policy = PolicyNetwork(input_size, 128, output_size)
optimizer = optim.Adam(policy.parameters(), lr=lr)
agent = REINFORCEAgent(policy, optimizer)

num_episodes = 1000 # TODO

for episode in range(num_episodes):
    state, info = env.reset()
    total_reward = 0

    # TODO
    # collect rewards and log probabilities for updating the policy in a loop
    while True:
        action = agent.select_action(state)
        next_state, reward, terminated, truncated, info = env.step(action)
        agent.store_reward(reward)
        total_reward += reward
        state = next_state

        if terminated or truncated:
            break

    agent.update_policy()
    if episode % 50 == 0:
        print(f'Episode {episode+1}: Total Reward = {total_reward}')
env.close()

Episode 1: Total Reward = 11.0
Episode 51: Total Reward = 28.0
Episode 101: Total Reward = 247.0
Episode 151: Total Reward = 196.0
Episode 201: Total Reward = 500.0
Episode 251: Total Reward = 288.0
Episode 301: Total Reward = 182.0
Episode 351: Total Reward = 187.0
Episode 401: Total Reward = 500.0
Episode 451: Total Reward = 500.0
Episode 501: Total Reward = 119.0
Episode 551: Total Reward = 500.0
Episode 601: Total Reward = 500.0
Episode 651: Total Reward = 500.0
Episode 701: Total Reward = 104.0
Episode 751: Total Reward = 110.0
Episode 801: Total Reward = 500.0
Episode 851: Total Reward = 500.0
Episode 901: Total Reward = 500.0
Episode 951: Total Reward = 462.0


## Evaluation **(5 points)**

Here we use the trained agent and collect a trajectory using it's policy. Calculate the cumulative reward by adding rewards in each time space. Save and display the video of this run in the end.

In [ ]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
state, _ = env.reset()
frames = []

total_reward = 0
# TODO
# run the policy in the environment in a loop
while True:
    frames.append(env.render())
    state = torch.from_numpy(state).float().unsqueeze(0)
    with torch.no_grad():
        action_probs = agent.policy(state)
    action = torch.argmax(action_probs, dim=1).item()  # Select the action with the highest probability
    state, reward, terminated, truncated, _ = env.step(action)
    total_reward += reward

    if terminated or truncated:
        break

env.close()
print(f'Total Reward: {total_reward}')

imageio.mimsave('./eval_reinforce.mp4', frames, fps=25)
show_video('./eval_reinforce.mp4')

Total Reward: 500.0


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


# Proximal Policy Optimization **(60 points)**

## Setup

## Explore the environment

This code is essential for rendering MUJOCO based environments.

In [ ]:
# Configure MuJoCo to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

env: MUJOCO_GL=egl


We will train a PPO agent in the `HalfCheetah` environment. This environment features continuous actions and more complex mechanics.

Explore this environment using random actions as well, and display the video of the resulting trajectory.

* What are the observation and action spaces of this environment?

* Are values bounded?

In [ ]:
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
env.reset()
frames = []

for _ in range(100):
    frames.append(env.render())
    action = env.action_space.sample() # TODO
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        break
env.close()
imageio.mimsave('./HalfCheetah.mp4', frames, fps=25)
show_video('./HalfCheetah.mp4')


## Actor & Critic **(15 points)**

Proximal Policy Optimization (PPO) is an advanced reinforcement learning algorithm that uses separate actor and critic networks to optimize policy performance.

The actor network is responsible for predicting a probability distribution over actions (discrete) or estimating the value for each action (continuous), given the current state, while the critic network evaluates how good the action taken by the actor is, by predicting the reward based on state.


In [ ]:
class Actor(nn.Module):
    def __init__(self, state_dim, hidden_size, action_dim):
        super(Actor, self).__init__()
        # TODO
        # Define the Actor architecture
        self.fc1 = nn.Linear(state_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.mu_head = nn.Linear(hidden_size, action_dim)
        self.std_head = nn.Linear(hidden_size, action_dim)

    def forward(self, state):
        # TODO
        # In case of continuous environment, we usually
        # predict a mean and std for each action and sample
        # the action from a normal distribution
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        mu = torch.tanh(self.mu_head(x))
        std = F.softplus(self.std_head(x))

        return mu, std

class Critic(nn.Module):
    def __init__(self, state_dim, hidden_size):
        super(Critic, self).__init__()
        # TODO
        # Define the Critic architecture
        self.fc1 = nn.Linear(state_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.value_head = nn.Linear(hidden_size, 1)

    def forward(self, state):
        # TODO
        # Predict the value of state
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        value = self.value_head(x)

        return value


## Memory

PPO algorithms need to store sequences of actions, states, log probabilities, rewards, and state values to train the agent. This data is captured in the `Memory` class, which facilitates batch processing by holding and then clearing these elements at the end of each training iteration.

In [ ]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.state_values = []

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]


## Agent **(35 points)**

In PPO, the actor's goal is to maximize the expected return. However, direct maximization can cause large policy updates, risking instability. To prevent this, PPO employs a clipping mechanism, limiting policy changes to a defined range.

$$ L^{CLIP}(\theta) = \hat{\mathbb{E}}_t \left[ \min(r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t) \right]
 $$

Additionally, it uses a probability ratio to scale updates, ensuring changes This ratio provides a scaling factor for the policy updates, ensuring that changes are made in proportion to the improvement in policy performance.

$$ r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}
 $$

 The critic aims to minimize the error between its predictions and the actual returns.

 $$ L^{VF}(\phi) = \left( V_\phi(s_t) - \hat{R}_t \right)^2
 $$

In [ ]:
class PPO(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=64, lr=1e-4, gamma=0.99, epochs=4, eps_clip=0.2):
        super(PPO, self).__init__()
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.epochs = epochs

        self.actor = Actor(state_dim, hidden_size, action_dim)
        self.critic = Critic(state_dim, hidden_size)

        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=lr)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=lr)
        self.memory = Memory()

    def select_action(self, state):
        # TODO
        # Save state, action, log probability and state value of current step in the memory buffer.
        # predict the actions by sampling from a normal distribution
        # based on the mean and std calculated by actor
        state = torch.FloatTensor(state).unsqueeze(0)
        mu, std = self.actor(state)
        dist = Normal(mu, std)
        action = dist.sample()
        action_logprob = dist.log_prob(action).sum(dim=-1)

        self.memory.states.append(state)
        self.memory.actions.append(action)
        self.memory.logprobs.append(action_logprob)
        self.memory.state_values.append(self.critic(state))

        action =  action.detach().numpy().flatten()

        return action

    def evaluate(self, state, action):
        # TODO
        # evaluate the state value of this state and log probability of choosing this
        mu, std = self.actor(state)
        dist = Normal(mu, std)
        action_logprobs = dist.log_prob(action).sum(dim=-1)
        state_value = self.critic(state)
        entropy = dist.entropy().sum(dim=-1)

        return action_logprobs, state_value, entropy

    def update(self):
        rewards = []
        discounted_reward = 0
        # TODO
        # Calculate discounted rewards
        for reward in reversed(self.memory.rewards):
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)

        rewards = torch.tensor(rewards, dtype=torch.float32)

        # TODO
        # load saved states, actions, log probs, and state
        old_states = torch.cat(self.memory.states).detach()
        old_actions = torch.cat(self.memory.actions).detach()
        old_logprobs = torch.cat(self.memory.logprobs).detach()
        old_state_values = torch.cat(self.memory.state_values).detach().squeeze()


        # TODO
        # Calculate advantages for each timestep (usually difference of rewards and state values)
        advantages = rewards - old_state_values # TODO

        loss_ac = 0
        loss_cri = 0
        for _ in range(self.epochs):
            # calculate logprobs and state values based on the new policy
            logprobs, state_values, entropy = self.evaluate(old_states, old_actions) # TODO

            ratios = torch.exp(logprobs - old_logprobs)
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * advantages

            # TODO
            # Calculate the loss function and perform the optimization

            loss_actor = -torch.min(surr1, surr2).mean() - 0.01 * entropy.mean() # TODO
            loss_critic = F.mse_loss(state_values, rewards.unsqueeze(1)) # TODO

            self.optimizer_actor.zero_grad()
            loss_actor.backward()
            loss_ac += loss_actor.item()
            self.optimizer_actor.step()

            self.optimizer_critic.zero_grad()
            loss_critic.backward()
            loss_cri += loss_critic.item()
            self.optimizer_critic.step()

        # clear the buffer
        self.memory.clear()
        return loss_ac, loss_cri

## Training **(5 points)**

Define the hyperparameters and complete the training loop.

In [ ]:
env = gym.make("HalfCheetah-v4")
state_dim = env.observation_space.shape[0] # TODO
action_dim = env.action_space.shape[0] # TODO
hidden_size = 64 # TODO
lr = 3e-4 # TODO

agent = model = PPO(state_dim, action_dim, hidden_size=hidden_size, lr=lr)

# We need to train for many more steps to achieve acceptable results compared to the last environment
num_episodes = 10000 # TODO

actor_losses = []
critic_losses = []
moving_rewards = np.array([])

for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    # TODO
    # write the training
    while True:
        action = agent.select_action(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        agent.memory.rewards.append(reward)
        state = next_state
        total_reward += reward

        if terminated or truncated:
            break

    loss_ac, loss_cri = agent.update()
    actor_losses.append(loss_ac)
    critic_losses.append(loss_cri)
    moving_rewards = np.append(moving_rewards, total_reward)
    if episode % 100 == 0:
        print(f"actor loss:\t{loss_ac:.6f}")
        print(f"critic loss:\t{loss_cri:.6f}")
        print(f'Episode {episode}: Going Reward = {moving_rewards.mean():.1f}: Std = {moving_rewards.std():.1f}')
        moving_rewards = np.array([])

env.close()

actor loss:	201.844334
critic loss:	11794.062256
Episode 0: Going Reward = -566.0: Std = 0.0
actor loss:	-52.588943
critic loss:	1235.915558
Episode 100: Going Reward = -344.3: Std = 102.1
actor loss:	-5.075188
critic loss:	1137.989136
Episode 200: Going Reward = 25.0: Std = 169.3
actor loss:	-79.733889
critic loss:	4636.339722
Episode 300: Going Reward = 420.0: Std = 255.9
actor loss:	-36.170876
critic loss:	12770.718262
Episode 400: Going Reward = 537.4: Std = 342.8
actor loss:	38.967701
critic loss:	9219.456787
Episode 500: Going Reward = 781.4: Std = 581.1
actor loss:	192.996590
critic loss:	14085.101562
Episode 600: Going Reward = 1490.0: Std = 1014.2
actor loss:	-30.540088
critic loss:	11860.225098
Episode 700: Going Reward = 2009.4: Std = 900.8
actor loss:	-5.465752
critic loss:	23575.567383
Episode 800: Going Reward = 2406.6: Std = 910.4
actor loss:	23.407373
critic loss:	3500.728821
Episode 900: Going Reward = 2041.3: Std = 1353.3
actor loss:	64.860647
critic loss:	10976.69921

## Evaluation **(5 points)**

Evaluate the trained policy on the environment. Calculate the cumulative reward and display the video of the trajectory.

In [ ]:
env = gym.make("HalfCheetah-v4", render_mode="rgb_array")
state, _ = env.reset()
frames = []

total_reward = 0
# TODO
# run the policy in the environment in a loop
while True:
    action = agent.select_action(state)
    next_state, reward, terminated, truncated, _ = env.step(action)
    frames.append(env.render())
    total_reward += reward
    state = next_state

    if terminated or truncated:
        break

env.close()
print(f'Total Reward: {total_reward}')

imageio.mimsave('./eval_ppo.mp4', frames, fps=25)
show_video('./eval_ppo.mp4')

Total Reward: 4462.972174087664
